In [1]:
import os
import json
import pandas as pd
import traceback

In [2]:
from langchain import HuggingFaceHub

In [3]:
import os
from dotenv import load_dotenv
load_dotenv()
os.environ['HUGGINGFACEHUB_API_TOKEN'] = os.getenv("HUGGINGFACEHUB_API_TOKEN")

In [4]:
from transformers import T5ForConditionalGeneration, AutoTokenizer, pipeline
import torch
model = T5ForConditionalGeneration.from_pretrained("google/flan-ul2")
tokenizer = AutoTokenizer.from_pretrained("google/flan-ul2")

device = torch.device('cpu')
model.to(device)

/home/user/Meet Patel/MCQ_Generator/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 8/8 [00:07<00:00,  1.08it/s]
/home/user/Meet Patel/MCQ_Generator/venv/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


T5ForConditionalGeneration(
  (shared): Embedding(32128, 4096)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 4096)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=4096, out_features=4096, bias=False)
              (k): Linear(in_features=4096, out_features=4096, bias=False)
              (v): Linear(in_features=4096, out_features=4096, bias=False)
              (o): Linear(in_features=4096, out_features=4096, bias=False)
              (relative_attention_bias): Embedding(32, 16)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=4096, out_features=16384, bias=False)
              (wi_1): Linear(in_features=4096, out_features=16384, bias=False)
     

In [5]:
# llm = HuggingFaceHub(
#     repo_id = 'google/flan-ul2', 
#     # repo_id = 'google/flan-t5-large',
#     model_kwargs={"temperature": 0.7}
# )
from langchain.llms import HuggingFacePipeline
llm = pipeline(
    task="text2text-generation",
    model=model,
    tokenizer=tokenizer,
    device=-1  # -1 indicates CPU
    # pipeline_kwargs={"max_new_tokens": 50, "num_return_sequences": 1, "num_beams": 1, "early_stopping": True},
)

In [6]:
llm

In [7]:
from langchain import PromptTemplate
from langchain.chains import LLMChain, SequentialChain
# from langchain_huggingface import HuggingFaceHubCallbackHandler
import PyPDF2

In [8]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [9]:
TEMPLATE = """
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz of {number} multiple choice questions for {subject} students in {tone} tone.
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like RESPONSE_JSON below and use it as guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [10]:
# Designing the Prompt template
quiz_generation_prompt = PromptTemplate(
    input_variables = ["text", "number", "subject", "tone", "response_json"],
    template = TEMPLATE
)

In [11]:
quiz_chain = LLMChain(
    llm = HuggingFacePipeline(pipeline=llm),
    prompt = quiz_generation_prompt,
    output_key = "quiz",
    verbose = True
)

/home/user/Meet Patel/MCQ_Generator/venv/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFacePipeline`.
  warn_deprecated(
/home/user/Meet Patel/MCQ_Generator/venv/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [12]:
TEMPLATE2="""
You are an expert english grammarin and writer: Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis.
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student's abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [13]:
quiz_evaluation_prompt = PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE2)

In [14]:
review_chain = LLMChain(
    llm = HuggingFacePipeline(pipeline = llm), 
    prompt = quiz_evaluation_prompt, 
    output_key = "review", 
    verbose = True
)

In [15]:
generate_evaluate_chain = SequentialChain(
    chains=[quiz_chain, review_chain],
    input_variables=["text", "number", "subject", "tone", "response_json"],
    output_variables=["quiz", "review"],
    verbose=True,
)

In [16]:
file_path = '/home/user/Meet Patel/MCQ_Generator/data.txt'
with open(file_path, 'r') as file:
    TEXT = file.read()

TEXT

'Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]\n\nBiologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each defined by the nature of their research questions and the tools that they use.[7][8][9] Like other scientists, 

In [17]:
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [18]:
NUMBER = 5
SUBJECT = "Biology"
TONE = "simple"

In [28]:
#https://python.langchain.com/docs/modules/model_io/llms/token_usage_tracking

#How to setup Token Usage Tracking in LangChain
# with get_openai_callback() as cb:
response=generate_evaluate_chain(
    {
        "text": TEXT,
        "number": NUMBER,
        "subject":SUBJECT,
        "tone": TONE,
        "response_json": RESPONSE_JSON
    }
)

# print(f"Total Tokens:{cb.total_tokens}")
# print(f"Prompt Tokens:{cb.prompt_tokens}")
# print(f"Completion Tokens:{cb.completion_tokens}")
# print(f"Total Cost:{cb.total_cost}")



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]

Biologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each define

/home/user/Meet Patel/MCQ_Generator/venv/lib/python3.12/site-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(



> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:

You are an expert english grammarin and writer: Given a Multiple Choice Quiz for Biology students.You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis.
if the quiz is not at per with the cognitive and analytical abilities of the students,update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student's abilities
Quiz_MCQs:
What is the scientific study of life?

Check from an expert English Writer of the above quiz:



/home/user/Meet Patel/MCQ_Generator/venv/lib/python3.12/site-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(



> Finished chain.

> Finished chain.


In [35]:
response

{'text': 'Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]\n\nBiologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each defined by the nature of their research questions and the tools that they use.[7][8][9] Like other sci

In [33]:
quiz = response.get("quiz")
quiz

'What is the scientific study of life?'

In [32]:
# Decode the JSON
try:
    quiz_json = json.loads(quiz)
    print("Decoded JSON:", quiz_json)
except json.JSONDecodeError as e:
    print("JSON decoding failed:", e)

JSON decoding failed: Expecting value: line 1 column 1 (char 0)


In [ ]:
quiz_table_data = []
for key, value in quiz_json.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [ ]:
quiz_table_data

In [ ]:
quiz = pd.DataFrame(quiz_table_data)

In [ ]:
quiz.to_csv("machinelearning.csv",index=False)

In [ ]:
# Try with Other model and Data